In [7]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
import json # library to handle JSON files
import bs4 as bs
import urllib.request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 


# geocoder
!conda install -c conda-forge geocoder --yes 
import geocoder

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# use the inline backend to generate the plots within the browser
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot') # optional: for ggplot-like style

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0
print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

# All requested packages already installed.

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

# All requested packages already installed.

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.

In [25]:
# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [98]:
df = pd.DataFrame(l, columns=["Postal Code", "Borough", "Neighbourhood"])
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [99]:
df1 = df[df.Borough != 'Not assigned']
df1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [100]:
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)
df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [102]:
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])
df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [103]:
df3 = df2[df2['Borough'].str.contains('Toronto',regex=False)]
df3.head()

,Postal Code,Borough,Neighbourhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
15,M5C,Downtown Toronto,St. James Town
19,M4E,East Toronto,The Beaches


In [104]:
print('The dataframe contains {} boroughs and {} neighborhoods.'.format(
        len(df3['Borough'].unique()),df3.shape[0]))

The dataframe contains 5 boroughs and 40 neighborhoods.


In [106]:
print(df3.groupby('Borough').count()['Neighbourhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
Toronto/York         1
West Toronto         6
Name: Neighbourhood, dtype: int64


In [107]:
long_lat = pd.read_csv('https://cocl.us/Geospatial_data')
long_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [108]:
df4 = pd.merge(df3,long_lat, on = "Postal Code")
df4.tail()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
35,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846
36,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
37,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
38,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
39,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [109]:
!conda install -c conda-forge geopy --yes                 
from geopy.geocoders import Nominatim                    # module for geocoding
!conda install -c conda-forge folium=0.5.0 --yes         
import folium                                            # module to create leaflet maps

import matplotlib as plt                                 # visualization package
import matplotlib.cm as cm                               # visualization package with colour mapping features
import matplotlib.colors as colors                       # visualization package to display various colors
from sklearn.cluster import KMeans                       # module used to cluster data points

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

# All requested packages already installed.

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

# All requested packages already installed.



In [110]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="TO")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [112]:
map_toronto = folium.Map(location=[43.6534817,-79.3839347],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=4,
    popup=label,
    color='red',
    fill=True,
    fill_color='white',
    fill_opacity=.7,
    parse_html=False).add_to(map_toronto)

map_toronto

In [113]:
df4.Borough.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Toronto/York'], dtype=object)

In [115]:
k=4

toronto_clustering = df4.drop(['Postal Code','Borough','Neighbourhood'],axis=1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_

array([0, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 1, 3, 0, 1, 3, 0, 3, 2, 2, 1, 2,
       2, 1, 2, 0, 1, 2, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 3], dtype=int32)

In [116]:
toronto_cluster = df4.copy()
toronto_cluster["Borough ID"] = kmeans.labels_
toronto_cluster.sort_values(by='Borough')

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Borough ID
19,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2
21,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2
22,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2
24,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,2
25,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0
27,M4S,Central Toronto,Davisville,43.704324,-79.388790,2
30,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2
32,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,2
18,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2
31,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0


In [118]:
# create map
map_clusters = folium.Map(location=[43.6534817,-79.3839347],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.Dark2(np.linspace(0, 1, len(ys)))
palette = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map while each cluster/borough is defined by a color
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto_cluster['Latitude'], toronto_cluster['Longitude'], toronto_cluster['Neighbourhood'], toronto_cluster['Borough ID']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=palette[cluster-1],
        fill=True,
        fill_color=palette[cluster-1],
        fill_opacity=.7, 
        parse_html=False).add_to(map_clusters)
    
map_clusters

In [119]:
import json                                        # library to handle JSON files
from pandas.io.json import json_normalize          # package that tranforms JSON file into a pandas dataframe

In [126]:
CLIENT_ID = 'BSGKIURX4BRKJQVKKMXK4NO1M50GDTEAAW4YUOGN5PECKIQ5' #Foursquare ID
CLIENT_SECRET = 'UZCWBSTUHYOJB3KIWRNS0BOWWL0IJ3HMKLUXX2CVUJQMTU2C' #Foursquare Secret
ACCESS_TOKEN = 'QFW3ZH5O0KW1RMC1PYGDFQJ2DZ5QNOEIUDYXNRUGD0UT1DKI' #FourSquare Access Token
VERSION = '20180604'


In [127]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="TO_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [139]:
radius = 1000
LIMIT = 100

def getVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

# create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
# make the GET request
        venue_results = requests.get(url).json()['response']['groups'][0]['items']
        
# return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood_Latitude', 
                  'Neighbourhood_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [140]:
to_venues = getVenues(names=toronto_cluster['Neighbourhood'],
                            latitudes=toronto_cluster['Latitude'],
                            longitudes=toronto_cluster['Longitude'])
to_venues

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
5,"Regent Park, Harbourfront",43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
6,"Regent Park, Harbourfront",43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
7,"Regent Park, Harbourfront",43.654260,-79.360636,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
8,"Regent Park, Harbourfront",43.654260,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Underpass Park,43.655764,-79.354806,Park


In [162]:
tor_venues = to_venues[to_venues['Venue_Category'].str.contains('staurant')]
tor_venues.head()

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
12,"Regent Park, Harbourfront",43.65426,-79.360636,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
14,"Regent Park, Harbourfront",43.65426,-79.360636,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
21,"Regent Park, Harbourfront",43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
31,"Regent Park, Harbourfront",43.65426,-79.360636,Souvlaki Express,43.655584,-79.364438,Greek Restaurant


In [142]:
print('There are {} restaurants in the city of Toronto.'.format(
    tor_venues.Venue_Category.count()))

There are 832 restaurants in the city of Toronto.


In [143]:
print('There are {} uniques restaurant cuisines.'.format(len(tor_venues['Venue_Category'].unique())))

There are 53 uniques restaurant cuisines.


In [144]:
IndTO = tor_venues.loc[tor_venues['Venue_Category'] == 'Indian Restaurant']
IndTO.head()

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
80,"Regent Park, Harbourfront",43.654260,-79.360636,Bombay Palace,43.651174,-79.372140,Indian Restaurant
94,"Regent Park, Harbourfront",43.654260,-79.360636,Sultan Of Samosas,43.660227,-79.366226,Indian Restaurant
139,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant
430,The Beaches,43.676357,-79.293031,Delhi Bistro,43.672506,-79.288353,Indian Restaurant
684,Christie,43.669542,-79.422564,Banjara Indian Cuisine,43.662916,-79.421911,Indian Restaurant


In [145]:
noIndTO = tor_venues.loc[tor_venues['Venue_Category'] != 'Indian Restaurant']
noIndTO.head()

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
12,"Regent Park, Harbourfront",43.65426,-79.360636,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
14,"Regent Park, Harbourfront",43.65426,-79.360636,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
21,"Regent Park, Harbourfront",43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
31,"Regent Park, Harbourfront",43.65426,-79.360636,Souvlaki Express,43.655584,-79.364438,Greek Restaurant


In [146]:
# one hot encoding
to_onehot = pd.get_dummies(tor_venues[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column into dataframe
to_onehot['Neighbourhood'] = tor_venues['Neighbourhood'] 

# create dataframe with venue common mean by Neighborhood
to_rank = to_onehot.groupby('Neighbourhood').mean().reset_index()
print(to_rank.shape)
to_rank

(38, 54)


,Neighbourhood,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,South American Restaurant,Spanish Restaurant,Sushi Restaurant,Syrian Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.058824,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.235294,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.176471,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.038462,0.000000,0.000000,0.000000,0.0,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.076923,0.000000,0.076923,0.038462,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.192308,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.115385,0.000000,0.076923,0.000000
2,"Business reply mail Processing Centre, South C...",0.100000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.043478,0.043478,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.130435,0.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.130435,0.043478,0.086957,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.086957,0.000000
4,Christie,0.029412,0.000000,0.000000,0.000000,0.0,0.000000,0.029412,0.029412,0.000000,0.000000,0.000000,0.000000,0.029412,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.029412,0.029412,0.029412,0.382353,0.029412,0.000000,0.088235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.029412,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.029412
5,Church and Wellesley,0.034483,0.000000,0.000000,0.000000,0.0,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.172414,0.137931,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.068966,0.000000,0.000000,0

In [148]:
num_top_venues = 5

for hood in to_rank['Neighbourhood']:
    print("----"+hood+"----")
    temp = to_rank[to_rank['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0            Japanese Restaurant  0.24
1                     Restaurant  0.18
2             Seafood Restaurant  0.12
3              French Restaurant  0.06
4  Vegetarian / Vegan Restaurant  0.06


----Brockton, Parkdale Village, Exhibition Place----
                           venue  freq
0                     Restaurant  0.19
1             Tibetan Restaurant  0.12
2  Vegetarian / Vegan Restaurant  0.08
3             Italian Restaurant  0.08
4              Indian Restaurant  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Italian Restaurant   0.2
1  Fast Food Restaurant   0.2
2      Sushi Restaurant   0.2
3   American Restaurant   0.1
4       Thai Restaurant   0.1


----Central Bay Street----
                           venue  freq
0            Japanese Restaurant  0.13
1               Ramen Restaurant  0.13
2  Vegetarian / Vegan Restaurant  0.

In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [151]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighbourhood'] = to_rank['Neighbourhood']

for ind in np.arange(to_rank.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_rank.iloc[ind, :], num_top_venues)

venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Japanese Restaurant,Restaurant,Seafood Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,New American Restaurant,Middle Eastern Restaurant,Italian Restaurant,American Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Indian Restaurant,American Restaurant,Falafel Restaurant,Tapas Restaurant,Seafood Restaurant,New American Restaurant
2,"Business reply mail Processing Centre, South C...",Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,American Restaurant,Thai Restaurant,French Restaurant,Restaurant,Tapas Restaurant,Portuguese Restaurant,Mediterranean Restaurant
3,Central Bay Street,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Mexican Restaurant,Falafel Restaurant,Thai Restaurant,Restaurant,Modern European Restaurant
4,Christie,Korean Restaurant,Mexican Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,South American Restaurant,Restaurant,Ramen Restaurant,Latin American Restaurant
5,Church and Wellesley,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Thai Restaurant,Restaurant,Caribbean Restaurant,Ramen Restaurant,Indian Restaurant,Ethiopian Restaurant,Theme Restaurant
6,"Commerce Court, Victoria Hotel",Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,American Restaurant,Mediterranean Restaurant,Fast Food Restaurant,New American Restaurant
7,Davisville,Italian Restaurant,Sushi Restaurant,Restaurant,Indian Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Fast Food Restaurant,Japanese Restaurant,French Restaurant,Vegetarian / Vegan Restaurant
8,Davisville North,Italian Restaurant,Restaurant,Fast Food Restaurant,Sushi Restaurant,Thai Restaurant,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant
9,"Dufferin, Dovercourt Village",Italian Restaurant,Sushi Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Restaurant,Brazilian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Thai Restaurant


In [152]:
k_clusters = 4

to_fsquare_clustering = to_rank.drop('Neighbourhood', 1)

# run k-means clustering
kmeans1 = KMeans(init = "k-means++", n_clusters=k_clusters, random_state=None).fit(to_fsquare_clustering)

# check cluster labels generated for each row in the dataframe
kmeans1.labels_

array([2, 3, 0, 2, 3, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 3, 3, 3, 0, 0, 0, 0,
       0, 2, 1, 0, 0, 3, 2, 2, 2, 0, 0, 3, 3, 3, 2, 3], dtype=int32)

In [153]:
# add clustering labels
venues_sorted.insert(0, 'Cluster_ID', kmeans1.labels_ )

toronto_rank = noIndTO

# merge toronto_ with neighbourhoods to add latitude/longitude for each neighborhood
to_merged = df4.join(venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

to_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Thai Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Pakistani Restaurant,Greek Restaurant,German Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Restaurant,Ethiopian Restaurant,Mexican Restaurant,Modern European Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Japanese Restaurant,Restaurant,Italian Restaurant,Seafood Restaurant,Ramen Restaurant,Middle Eastern Restaurant,German Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Sushi Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2.0,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Japanese Restaurant,Latin American Restaurant,Thai Restaurant,New American Restaurant,Asian Restaurant,Middle Eastern Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3.0,Japanese Restaurant,Asian Restaurant,Caribbean Restaurant,Restaurant,Mexican Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Ramen Restaurant,Mediterranean Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2.0,Japanese Restaurant,Restaurant,Seafood Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,New American Restaurant,Middle Eastern Restaurant,Italian Restaurant,American Restaurant
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2.0,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Mexican Restaurant,Falafel Restaurant,Thai Restaurant,Restaurant,Modern European Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3.0,Korean Restaurant,Mexican Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,South American Restaurant,Restaurant,Ramen Restaurant,Latin American Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2.0,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Seafood Restaurant,Restaurant,Italian Restaurant,American Restaurant,Mediterranean Restaurant,Brazilian Restaurant,Vegetarian / Vegan Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0.0,Italian Restaurant,Sushi Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Restaurant,Brazilian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Thai Restaurant


In [154]:
to_merged.dropna(axis=0, how='any',inplace=True)
to_merged.reset_index(inplace=True, drop=True)
to_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Thai Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Pakistani Restaurant,Greek Restaurant,German Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Restaurant,Ethiopian Restaurant,Mexican Restaurant,Modern European Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Japanese Restaurant,Restaurant,Italian Restaurant,Seafood Restaurant,Ramen Restaurant,Middle Eastern Restaurant,German Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Sushi Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2.0,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Japanese Restaurant,Latin American Restaurant,Thai Restaurant,New American Restaurant,Asian Restaurant,Middle Eastern Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3.0,Japanese Restaurant,Asian Restaurant,Caribbean Restaurant,Restaurant,Mexican Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Ramen Restaurant,Mediterranean Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2.0,Japanese Restaurant,Restaurant,Seafood Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,New American Restaurant,Middle Eastern Restaurant,Italian Restaurant,American Restaurant
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2.0,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Mexican Restaurant,Falafel Restaurant,Thai Restaurant,Restaurant,Modern European Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3.0,Korean Restaurant,Mexican Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,South American Restaurant,Restaurant,Ramen Restaurant,Latin American Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2.0,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Seafood Restaurant,Restaurant,Italian Restaurant,American Restaurant,Mediterranean Restaurant,Brazilian Restaurant,Vegetarian / Vegan Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0.0,Italian Restaurant,Sushi Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Restaurant,Brazilian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Thai Restaurant


In [156]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.Dark2(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster_ID']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)


for lat, lon, poi in zip(MexTO['Venue_Latitude'], IndTO['Venue_Longitude'], IndTO['Neighbourhood']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=1).add_to(map_clusters)    
    

map_clusters

In [157]:
GrayPts = to_merged.loc[to_merged['Cluster_ID'] == 0, to_merged.columns[[1] + list(range(2, to_merged.shape[1]))]]
GrayPts

,Borough,Neighbourhood,Latitude,Longitude,Cluster_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Thai Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Pakistani Restaurant,Greek Restaurant,German Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Restaurant,Ethiopian Restaurant,Mexican Restaurant,Modern European Restaurant
9,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0.0,Italian Restaurant,Sushi Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Restaurant,Brazilian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Thai Restaurant
17,East Toronto,Studio District,43.659526,-79.340923,0.0,American Restaurant,Vietnamese Restaurant,Italian Restaurant,French Restaurant,Sushi Restaurant,Thai Restaurant,Latin American Restaurant,Restaurant,Seafood Restaurant,Falafel Restaurant
18,Central Toronto,Roselawn,43.711695,-79.416936,0.0,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
20,Central Toronto,Davisville North,43.712751,-79.390197,0.0,Italian Restaurant,Restaurant,Fast Food Restaurant,Sushi Restaurant,Thai Restaurant,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant
21,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0.0,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Asian Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Persian Restaurant,Fast Food Restaurant,Restaurant,Vegetarian / Vegan Restaurant
22,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0.0,Thai Restaurant,Sushi Restaurant,Italian Restaurant,Mexican Restaurant,Cajun / Creole Restaurant,Vietnamese Restaurant,Indian Restaurant,Restaurant,Fast Food Restaurant,Seafood Restaurant
23,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0.0,Italian Restaurant,Restaurant,Mexican Restaurant,Thai Restaurant,Vietnamese Restaurant,Chinese Restaurant,Sushi Restaurant,Ramen Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant
25,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0.0,Sushi Restaurant,Thai Restaurant,Eastern European Restaurant,American Restaurant,Restaurant,Cuban Restaurant,Mediterranean Restaurant,Italian Restaurant,Falafel Restaurant,Vietnamese Restaurant


In [158]:
GreenPts = to_merged.loc[to_merged['Cluster_ID'] == 1, to_merged.columns[[1] + list(range(2, to_merged.shape[1]))]]
GreenPts

,Borough,Neighbourhood,Latitude,Longitude,Cluster_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,Rosedale,43.679563,-79.377529,1.0,Japanese Restaurant,Filipino Restaurant,Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Pakistani Restaurant


In [159]:
PurplePts = to_merged.loc[to_merged['Cluster_ID'] == 2, to_merged.columns[[1] + list(range(2, to_merged.shape[1]))]]
PurplePts

,Borough,Neighbourhood,Latitude,Longitude,Cluster_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2.0,Japanese Restaurant,Restaurant,Italian Restaurant,Seafood Restaurant,Ramen Restaurant,Middle Eastern Restaurant,German Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Sushi Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,2.0,Restaurant,Italian Restaurant,Seafood Restaurant,American Restaurant,Japanese Restaurant,Latin American Restaurant,Thai Restaurant,New American Restaurant,Asian Restaurant,Middle Eastern Restaurant
5,Downtown Toronto,Berczy Park,43.644771,-79.373306,2.0,Japanese Restaurant,Restaurant,Seafood Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,New American Restaurant,Middle Eastern Restaurant,Italian Restaurant,American Restaurant
6,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2.0,Japanese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Mexican Restaurant,Falafel Restaurant,Thai Restaurant,Restaurant,Modern European Restaurant
8,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2.0,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Seafood Restaurant,Restaurant,Italian Restaurant,American Restaurant,Mediterranean Restaurant,Brazilian Restaurant,Vegetarian / Vegan Restaurant
10,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2.0,Japanese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Italian Restaurant,Indian Restaurant,French Restaurant,Seafood Restaurant,Mediterranean Restaurant,Chinese Restaurant
13,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,2.0,Japanese Restaurant,Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,American Restaurant,Mediterranean Restaurant,Brazilian Restaurant
16,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,2.0,Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,American Restaurant,Mediterranean Restaurant,Fast Food Restaurant,New American Restaurant
33,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,2.0,Restaurant,Japanese Restaurant,Seafood Restaurant,American Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Comfort Food Restaurant,New American Restaurant,Thai Restaurant
34,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2.0,Japanese Restaurant,Thai Restaurant,Italian Restaurant,Indian Restaurant,Caribbean Restaurant,Taiwanese Restaurant,Restaurant,Ramen Restaurant,Mediterranean Restaurant,Mexican Restaurant


In [160]:
YellowPts = to_merged.loc[to_merged['Cluster_ID'] == 3, to_merged.columns[[1] + list(range(2, to_merged.shape[1]))]]
YellowPts

,Borough,Neighbourhood,Latitude,Longitude,Cluster_ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,The Beaches,43.676357,-79.293031,3.0,Japanese Restaurant,Asian Restaurant,Caribbean Restaurant,Restaurant,Mexican Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Ramen Restaurant,Mediterranean Restaurant
7,Downtown Toronto,Christie,43.669542,-79.422564,3.0,Korean Restaurant,Mexican Restaurant,Ethiopian Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,South American Restaurant,Restaurant,Ramen Restaurant,Latin American Restaurant
11,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,3.0,Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Japanese Restaurant,Seafood Restaurant,Vietnamese Restaurant,American Restaurant,Thai Restaurant,Korean Restaurant
12,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3.0,Greek Restaurant,Fast Food Restaurant,Italian Restaurant,Ramen Restaurant,Restaurant,New American Restaurant,Caribbean Restaurant,Chinese Restaurant,Tibetan Restaurant,Tapas Restaurant
14,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,3.0,Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Indian Restaurant,American Restaurant,Falafel Restaurant,Tapas Restaurant,Seafood Restaurant,New American Restaurant
15,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3.0,Indian Restaurant,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Asian Restaurant,Italian Restaurant,Indian Chinese Restaurant,Pakistani Restaurant,Halal Restaurant,French Restaurant
19,Toronto/York,"Runnymede, The Junction, Weston-Pellam Park, C...",43.673185,-79.487262,3.0,Vietnamese Restaurant,Fast Food Restaurant,Asian Restaurant,Indian Restaurant,Thai Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
24,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3.0,Italian Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Thai Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Indian Restaurant,American Restaurant,French Restaurant,Modern European Restaurant
27,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,3.0,Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,French Restaurant,Thai Restaurant,Sushi Restaurant,Ramen Restaurant,Persian Restaurant
30,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,3.0,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,French Restaurant,Dumpling Restaurant,Restaurant,Ramen Restaurant,Persian Restaurant,Filipino Restaurant


In [163]:
numb_of_neigh = tor_final[['Neighbourhood', 'Cluster Labels']].groupby('Cluster Labels').count()
pl = numb_of_neigh.T

NameError: name 'tor_final' is not defined